In [8]:
#importing python libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

# requesting for content of the website of given URL ,i.e., of SOC website

URL = "https://itc.gymkhana.iitb.ac.in/wncc/soc/"
r = requests.get(URL)

# parsing the content obtained from the GET request using lxml as parser 

soup = BeautifulSoup(r.content, 'lxml')

projects_name = []
mentors_list = []
domains_list = []
mentees_list = []
projects_link_list = []

# finding all the project from the SOC website

contents = soup.find_all(
    'div', class_='col-lg-4 col-6 mb-4 shuffle-item')

# looping through each project

for content in contents:
    
    # finding project name and appending it to the list of all project names
    projects_name.append(content.find('p').text)
    
    # finding website link of each project and appending it to the list of links of all project names
    project_link = "https://itc.gymkhana.iitb.ac.in" + content.a['href']
    projects_link_list.append(project_link)
    
    # requesting for the content of the website of the project (scraping that website) and parsing it
    project_request = requests.get(project_link)
    project_soup = BeautifulSoup(project_request.content, 'lxml')
    project_details = project_soup.find('div', class_='container-fluid')
    
    project_domain_list = []
    
    # finding the domain names of each project
    project_domains = project_details.find_all('h4', class_='text-primary text-center topics')
    
    # storing the domains of each project as text 
    for project_domain in project_domains:
        project_domain_list.append(project_domain.text)
    
    # removing the square brackets from the list elements
    domain_str = str(project_domain_list)
    domain_str = domain_str.strip('[]')
    
    # appending the list of domains of a particular project to the overall list of domains of all projects
    domains_list.append(domain_str)
    
    # extracting the number of mentees from the list (which is the last element of the list) and appending it to the list of the number of mentees and then removing that element
    project_mentors = project_details.find_all('p', class_='lead')
    mentees_list.append(project_mentors[-1].text)
    project_mentors.pop()
    
    project_mentor_list = []
    
    # storing the mentors of each project as text 
    for project_mentor in project_mentors:
        project_mentor_list.append(project_mentor.text)
    
    # removing the square brackets from the list elements
    names_str = str(project_mentor_list)
    names_str = names_str.strip('[]')
    
    # appending the list of mentors of a particular project to the overall list of mentors of all project
    mentors_list.append(names_str)

data = ({'Project':projects_name,
         'Domain':domains_list,
         'Mentor(s)':mentors_list,
         'Number of Mentees':mentees_list,
         'More info':projects_link_list})    

df = pd.DataFrame({'Project':projects_name, 'Domain':domains_list, 'Mentor(s)':mentors_list, 'Number of Mentees':mentees_list, 'More info':projects_link_list})


# removing the square brackets from all the list elements

df['Project'] =  df['Project'].apply(lambda x: x.replace('[','').replace(']',''))
df['Domain'] =  df['Domain'].apply(lambda x: x.replace('[','').replace(']',''))
df['Mentor(s)'] =  df['Mentor(s)'].apply(lambda x: x.replace('[','').replace(']',''))
df['Number of Mentees'] =  df['Number of Mentees'].apply(lambda x: x.replace('[','').replace(']',''))
df['More info'] =  df['More info'].apply(lambda x: x.replace('[','').replace(']',''))

# storing all the scraped data in a csv file

df.to_csv('SOC_Data.csv')
   
